In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

# 그간 배웠던 Sequential 모델
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 함수형 API로 만든 모델
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 컴파일
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 훈련을 위해 랜덤 넘파이 생성
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size = 128)

# 모델 평가
score = model.evalutate(x_train, y_train)


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text') # 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스이다. 입력 이름을 지정할 수 있다.
embedd_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input) # 입력을 크기가 64인 벡터의 시퀀스로 임베딩한다.
encoded_text = layers.LSTM(32)(embedd_text) # LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩한다.

question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question') # 질문도 동일한 과정을 거친다.(층 객체는 다르다)

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1) # 인코딩된 질문과 텍스트를 연결한다.

answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated) # 소프트맥스 분류기 추가 

model = Model([text_input, question_input], answer) # 모델 객체를 만들고 2개의 입력과 출력을 주입한다.
model.compile(optimizer = 'rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [3]:
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answers = to_categorical(answers) # 답은 정수가 아닌 원-핫 인코딩된 벡터이다.

model.fit([text, question], answers, epochs=10, batch_size=128) # 리스트 입력을 사용하여 학습한다.

model.fit({'text':text, 'question':question}, answers,
          epochs=10, batch_size=128) # 딕셔너리 입력을 사용하여 학습한다.(입력 이름을 지정했을 때만 사용 가능)

In [4]:
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x) # 출력층에 이름 지정 
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

In [ ]:
model.compile(optimizer = 'rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer = 'rmsprop',
              loss={'age':'mse',
                    'income':'categorical_crossentropy',
                    'gender':'binary_crossentropy'}) # 위와 동일(출력 층에 이름을 지정했을 때만 사용 가능)

In [ ]:
model.compile(optimizer = 'rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer = 'rmsprop',
              loss={'age':'mse',
                    'income':'categorical_crossentropy',
                    'gender':'binary_crossentropy'},
              loss_weights={'age':0.25,
                            'income':1.,
                            'gender':10.}) # 위와 동일(출력 층에 이름을 지정했을 때만 사용 가능)

In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64) # 위 3개가 넘파이 배열이라고 가정

model.fit(posts, {'age' : age_targets, 
                  'incomde' : income_targets, 
                  'gender' : gender_targets},
          epochs=10, batch_size=64) # 위와 동일(출력 층에 이름을 지정했을 때만 사용 가능)

In [5]:
from tensorflow.keras import layers
branch_a = layers.Conv2D(128, 1,
                         activation='relu', strides=2)(x) # 모든 가지는 동일한 스트라이드(2)를 사용한다. 출력 크기를 동일하게 만들어 하나로 합치기 위해서이다.
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b) # 이 기지에서는 두 번째 합성곱 층에서 스트라이드를 적용한다.

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation="relu")(branch_c) # 이 기지에서는 평균 풀링 층에서 스트라이드를 적용한다.

branch_d = layers.Conv2D(128, 1, activation="relu")(x)
branch_d = layers.Conv2D(128, 3, activation="relu")(branch_d)
branch_d = layers.Conv2D(128, 3, activation="relu", strides=2)(branch_d)
output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1) # 모든 가지의 출력을 연결하여 모듈의 출력을 만든다.

ValueError: Input 0 of layer conv2d is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: (None, 128)

In [ ]:
from tensorflow.keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, stride=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x) # y와 크기를 맞추기 위해 1x1 합성곱을 사용하여 원본 텐서 x를 다운샘플링한다.

y = layers.add([y, residual]) # 다운샘플링된 x를 출력 특성에 더한다.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

lstm = layers.LSTM(32) # LSTM 층 객체 하나를 만든다.
left_input = Input(shape=(None, 128)) # 모델의 왼쪽 가지를 구성한다. 입력은 크기가 128인 벡터의 가변 길이 시퀀스이다.
left_output = lstm(left_input)

right_input = Input(shape=(None, 128)) # 모델의 오른쪽 가지를 구성한다. 기존 층 객체를 호출하면 가중치를 재사용된다.
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1) # 맨 위에 분류기를 놓는다.
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets) # LSTM 층의 가중치는 양쪽 입력을 바탕으로 업데이트된다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks_list = [ # fit() 메서드의 callbakcs 매개변수를 사용하여 콜백의 리스트를 모델로 전달한다. 몇 개의 콜백이라도 전달할 수 있다.
    EarlyStopping( # 성능 향상이 멈추면 훈련을 중지한다.
        monitor='val_acc' # 모델의 검증 정확도를 모니터링한다.
        patience=1, # 1 에포크보다 더 길게 (즉 2에포크 동안) 정확도가 향상되지 않으면 훈련이 중지된다.
    ),
    ModelCheckpoint( # 에포크마다 현재 가중치를 저장한다.
        filepath='my_model.h5', # 모델 파일의 경로
        monitor='val_loss',
        save_best_only=True, # 이 두 매개변수는 val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않는다는 뜻. 훈련하는 동안 가장 좋은 모델이 저장된다.
    )
]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x, y,
          batch_size=32,
          callbakcs=callbacks_list,
          validation_data=(x_val, y_val)) # 콜백이 검증 손실과 검증 정확도를 모니터링하기 때문에 validation_data 매개변수에 검증 데이터를 전달해야 한다.

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

callbacks_list = [
    ReduceLROnPlateau(
        monitor = 'val_loss', # 모델의 검증  손실을 모니터링한다.
        factor=0.1, # 콜백이 호출될 때 학습률을 10배로 줄인다.
        patience=10, # 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출된다.
    )
]

model.fit(x, y,
          epochs = 10,
          batch_size=32,
          callbakcs=callbacks_list,
          validation_data=(x_val, y_val)) # 콜백이 검증 손실과 검증 정확도를 모니터링하기 때문에 validation_data 매개변수에 검증 데이터를 전달해야 한다.

In [7]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model): # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출된다.
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                    layer_outputs) # 각 층의 활성화 출력을 반환하는 Model 객체이다.
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError("Requires validation_data.")
        
        validation_sample = self.validation_data[0][0:1] # 검증 데이터의 첫 번째 샘플을 가져온다.
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb') # 배열을 디스크에 저장한다.
        np.savez(f, activations)
        f.close()
